In [ ]:
import numpy as np
import argparse
import time
import random
import os
from os import path as osp
from termcolor import colored
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.optim.lr_scheduler as lr_scheduler
from datasets.training_dataset import HomoAffTps_Dataset
from datasets.load_pre_made_dataset import PreMadeDataset
from utils_training.optimize_GLUNet_with_adaptive_resolution import train_epoch, validate_epoch
from models.our_models.GLUNet import GLUNet_model
from utils_training.utils_CNN import load_checkpoint, save_checkpoint, boolean_string
from tensorboardX import SummaryWriter
from utils.image_transforms import ArrayToTensor

In [ ]:
#Synthetic training dataset is already created and saved in disk ? default is False'
pre_loaded_training_dataset =False,

# 'path to directory containing original images for training if --pre_loaded_training_'
# 'dataset is False or containing the synthetic pairs of training images and their '
# 'corresponding flow fields if --pre_loaded_training_dataset is True'
training_data_dir = ""

# 'path to directory containing original images for validation if --pre_loaded_training_'
# 'dataset is False or containing the synthetic pairs of validation images and their '
# 'corresponding flow fields if --pre_loaded_training_dataset is True'
evaluation_data_dir = ""

snapshots ="./snapshots"

# 'path to pre-trained model'
pretrained=None,
                       
# Optimization parameters
# 'learning rate'
lr=0.0001

# 'momentum constant'
momentum=4e-4, 

# 'start epoch'
start_epoch=-1

# 'number of training epochs'
n_epoch =200

# 'training batch size'
batch-size =16

# 'number of parallel threads for dataloaders'
n_threads = 8

# 'weight decay constant'
weight-decay=4e-4

# 'div flow'
div_flow=1.0

# Pseudo-RNG seed'
seed=1986


In [ ]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# datasets, pre-processing of the images is done within the network function !
source_img_transforms = transforms.Compose([ArrayToTensor(get_float=False)])
target_img_transforms = transforms.Compose([ArrayToTensor(get_float=False)])

In [ ]:
if not pre_loaded_training_dataset:
    # training dataset, created on the fly at each epoch
    pyramid_param = [520] # means that we get the ground-truth flow field at this size
    train_dataset = HomoAffTps_Dataset(image_path=training_data_dir,
                                       csv_file=osp.join('datasets', 'csv_files',
                                                     'homo_aff_tps_train_DPED_CityScape_ADE.csv'),
                                       transforms=source_img_transforms,
                                       transforms_target=target_img_transforms,
                                       pyramid_param=pyramid_param,
                                       get_flow=True,
                                       output_size=(520, 520)   
    # validation dataset
    pyramid_param = [520]
    val_dataset = HomoAffTps_Dataset(image_path=evaluation_data_dir,
                                         csv_file=osp.join('datasets', 'csv_files',
                                                           'homo_aff_tps_test_DPED_CityScape_ADE.csv'),
                                         transforms=source_img_transforms,
                                         transforms_target=target_img_transforms,
                                         pyramid_param=pyramid_param,
                                         get_flow=True,
                                         output_size=(520, 520))
else:
        # If synthetic pairs were already created and saved to disk, run instead of 'train_dataset' the following.
        # and replace args.training_data_dir by the root to folders containing images/ and flow/

    flow_transform = transforms.Compose([ArrayToTensor()]) # just put channels first and put it to float
    train_dataset, _ = PreMadeDataset(root=training_data_dir,
                                      source_image_transform=source_img_transforms,
                                      target_image_transform=target_img_transforms,
                                      flow_transform=flow_transform,
                                      co_transform=None,
                                      split=1)  # only training
    _, val_dataset = PreMadeDataset(root=evaluation_data_dir,
                                        source_image_transform=source_img_transforms,
                                        target_image_transform=target_img_transforms,
                                        flow_transform=flow_transform,
                                        co_transform=None,
                                        split=0)  # only validation

In [ ]:
# Dataloader
train_dataloader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=True,
                              num_workers=n_threads)
val_dataloader = DataLoader(val_dataset,
                            batch_size=20,
                            shuffle=False,
                            num_workers=n_threads)

In [ ]:
# models
model = GLUNet_model(batch_norm=True, pyramid_type='VGG',
                         div=div_flow, evaluation=False,
                         consensus_network=False,
                         cyclic_consistency=True,
                         dense_connection=True,
                         decoder_inputs='corr_flow_feat',
                         refinement_at_all_levels=False,
                         refinement_at_adaptive_reso=True)
print(colored('==> ', 'blue') + 'GLU-Net created.')

In [ ]:
if pretrained:
    # reload from pre_trained_model
    model, optimizer, scheduler, start_epoch, best_val = load_checkpoint(model, optimizer, scheduler, filename=pretrained)
    # now individually transfer the optimizer parts...
    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.to(device)
    cur_snapshot = os.path.basename(os.path.dirname(pretrained))
else:
    if not os.path.isdir(snapshots):
        os.mkdir(snapshots)
    cur_snapshot = time.strftime('%Y_%m_%d_%H_%M')
    if not osp.isdir(osp.join(snapshots, cur_snapshot)):
        os.mkdir(osp.join(snapshots, cur_snapshot))
    with open(osp.join(snapshots, cur_snapshot, 'args.pkl'), 'wb') as f:
        pickle.dump(args, f)
    best_val = -1
    start_epoch = 0